# Data prepare
* Import
* load csv data
* explore data


### Import libraries

In [1]:
import io
import os
import h5py
import pandas as pd 
import gc
import sys
sys.path.append('../src')



### Load csv data

In [2]:
# reading metadata 
df_train = pd.read_csv('E:\\Mine\\Skin_cancer_detection\\isic-2024-challenge\\train-metadata.csv', low_memory=False) #, low_memory error was raised only in the train data
df_test = pd.read_csv('E:\\Mine\\Skin_cancer_detection\\isic-2024-challenge\\test-metadata.csv')
df_train.head()

,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455
2,ISIC_0015864,0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040
3,ISIC_0015902,0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998
4,ISIC_0024200,0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510


### Exploration & Preprocessing

In [32]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401059 entries, 0 to 401058
Data columns (total 55 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   isic_id                       401059 non-null  object 
 1   target                        401059 non-null  int64  
 2   patient_id                    401059 non-null  object 
 3   age_approx                    398261 non-null  float64
 4   sex                           389542 non-null  object 
 5   anatom_site_general           395303 non-null  object 
 6   clin_size_long_diam_mm        401059 non-null  float64
 7   image_type                    401059 non-null  object 
 8   tbp_tile_type                 401059 non-null  object 
 9   tbp_lv_A                      401059 non-null  float64
 10  tbp_lv_Aext                   401059 non-null  float64
 11  tbp_lv_B                      401059 non-null  float64
 12  tbp_lv_Bext                   401059 non-nul

In [8]:
df_test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   isic_id                      3 non-null      object 
 1   patient_id                   3 non-null      object 
 2   age_approx                   3 non-null      float64
 3   sex                          3 non-null      object 
 4   anatom_site_general          3 non-null      object 
 5   clin_size_long_diam_mm       3 non-null      float64
 6   image_type                   3 non-null      object 
 7   tbp_tile_type                3 non-null      object 
 8   tbp_lv_A                     3 non-null      float64
 9   tbp_lv_Aext                  3 non-null      float64
 10  tbp_lv_B                     3 non-null      float64
 11  tbp_lv_Bext                  3 non-null      float64
 12  tbp_lv_C                     3 non-null      float64
 13  tbp_lv_Cext             

In [6]:
df_train.describe()

,target,age_approx,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,...,color_variance_ratio,border_color_interaction,size_color_contrast_ratio,age_normalized_nevi_confidence,color_asymmetry_index,3d_volume_approximation,color_range,shape_color_consistency,border_length_ratio,age_size_symmetry_index
count,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,...,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000
mean,0.000980,58.026849,3.930827,19.974007,14.919247,28.281706,26.913015,34.786341,30.921279,54.653689,...,0.498923,9.907461,0.552745,0.771166,0.284147,9205.636333,16.137811,0.765472,1.222163,71.752783
std,0.031288,13.549664,1.743068,3.999489,3.529384,5.278676,4.482994,5.708469,4.829345,5.520849,...,0.366799,9.116371,0.251006,0.923576,0.242250,11369.317438,5.933356,0.566263,0.158164,54.374588
min,0.000000,5.000000,1.000000,-2.487115,-9.080269,-0.730989,9.237066,3.054228,11.846520,-1.574164,...,0.000000,0.000000,0.045937,0.000000,0.000000,138.617712,3.461981,0.000000,0.925410,4.685714
25%,0.000000,50.000000,2.840000,17.330821,12.469740,24.704372,23.848125,31.003148,27.658285,51.566273,...,0.264660,4.728710,0.393165,0.001807,0.148505,3902.214016,11.861529,0.406266,1.107961,38.022474
50%,0.000000,60.000000,3.370000,19.801910,14.713930,28.171570,26.701704,34.822580,30.804893,55.035632,...,0.430651,7.503427,0.502032,0.248666,0.230988,6174.385340,15.074170,0.651553,1.177520,56.482092
75%,0.000000,70.000000,4.380000,22.304628,17.137175,31.637429,29.679913,38.430298,33.963868,58.298184,...,0.657203,12.072591,0.640375,1.500171,0.355263,10176.871359,19.372208,0.998492,1.293070,86.844205
max,1.000000,85.000000,28.400000,48.189610,37.021680,54.306900,48.372700,58.765170,54.305290,105.875784,...,7.934576,100.000000,5.076897,9.417809,5.234940,517100.410138,67.211678,9.203239,2.634308,911.515938


In [15]:
# Sum the null values in each of the numerical columns
null_sum = df_train[num_cols].isnull().sum()

# Display the result
print(null_sum)


age_approx                     2798
clin_size_long_diam_mm            0
tbp_lv_A                          0
tbp_lv_Aext                       0
tbp_lv_B                          0
tbp_lv_Bext                       0
tbp_lv_C                          0
tbp_lv_Cext                       0
tbp_lv_H                          0
tbp_lv_Hext                       0
tbp_lv_L                          0
tbp_lv_Lext                       0
tbp_lv_areaMM2                    0
tbp_lv_area_perim_ratio           0
tbp_lv_color_std_mean             0
tbp_lv_deltaA                     0
tbp_lv_deltaB                     0
tbp_lv_deltaL                     0
tbp_lv_deltaLB                    0
tbp_lv_deltaLBnorm                0
tbp_lv_eccentricity               0
tbp_lv_minorAxisMM                0
tbp_lv_nevi_confidence            0
tbp_lv_norm_border                0
tbp_lv_norm_color                 0
tbp_lv_perimeterMM                0
tbp_lv_radial_color_std_max       0
tbp_lv_stdL                 

In [14]:
isic_id = df_train.iloc[:, 0]
X_train = df_train.iloc[:, 1:-1]
y_train = df_train.iloc[:, -1]

categorical = X_train.select_dtypes(include= ['object'])
numerical = X_train.select_dtypes(exclude= ['object'])
categorical.columns

Index(['patient_id', 'sex', 'anatom_site_general', 'image_type',
       'tbp_tile_type', 'tbp_lv_location', 'tbp_lv_location_simple',
       'attribution', 'copyright_license', 'lesion_id', 'iddx_full', 'iddx_1',
       'iddx_2', 'iddx_3', 'iddx_4', 'iddx_5', 'mel_mitotic_index'],
      dtype='object')

In [17]:
df_train["tbp_lv_location_simple"].value_counts()

tbp_lv_location_simple
Torso Back     121902
Torso Front     87770
Left Leg        53075
Right Leg       49953
Left Arm        36348
Right Arm       34209
Head & Neck     12046
Unknown          5756
Name: count, dtype: int64

In [18]:
df_train["tbp_lv_location"].value_counts()

tbp_lv_location
Torso Back Top Third       71112
Torso Front Top Half       63350
Torso Back Middle Third    46185
Left Leg - Lower           27428
Right Leg - Lower          25208
Torso Front Bottom Half    24360
Left Leg - Upper           23673
Right Leg - Upper          23034
Right Arm - Upper          22972
Left Arm - Upper           22816
Head & Neck                12046
Left Arm - Lower           11939
Right Arm - Lower          10636
Unknown                     5756
Torso Back Bottom Third     4596
Left Leg                    1974
Right Leg                   1711
Left Arm                    1593
Right Arm                    601
Torso Front                   60
Torso Back                     9
Name: count, dtype: int64

In [13]:
df_train["anatom_site_general"].value_counts()

anatom_site_general
posterior torso    121902
lower extremity    103028
anterior torso      87770
upper extremity     70557
head/neck           12046
Name: count, dtype: int64

#### Columns to drop
* image_type
* tbp_lv_location
* attribution
* copyright_license
* iddx_5

#### Cat columns to convert to OneHot encoding
* sex
* tbp_tile_type
* tbp_lv_location_simple
* anatom_site_general

In [4]:
from finalmetadata import preprocessing

preprocessing(df_train)
    

In [13]:
#Load and explore new csv data after preprocessing
final_df_train = pd.read_csv("E:\\Mine\\Skin_cancer_detection\\notebooks\\final_train_data.csv",  low_memory=False)
columns_to_drop = ['sex_nan', "tbp_lv_location_simple_Unknown", "anatom_site_general_nan"] 
final_df_train.drop(columns=columns_to_drop, inplace=True)
final_df_train.head()

,isic_id,target,patient_id,age_approx,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,...,tbp_lv_location_simple_Left Leg,tbp_lv_location_simple_Right Arm,tbp_lv_location_simple_Right Leg,tbp_lv_location_simple_Torso Back,tbp_lv_location_simple_Torso Front,anatom_site_general_anterior torso,anatom_site_general_head/neck,anatom_site_general_lower extremity,anatom_site_general_posterior torso,anatom_site_general_upper extremity
0,ISIC_0015670,0,IP_1235828,60.0,3.04,20.244422,16.261975,26.922447,23.954773,33.684638,...,0,0,1,0,0,0,0,1,0,0
1,ISIC_0015845,0,IP_8170065,60.0,1.10,31.712570,25.364740,26.331000,24.549290,41.219030,...,0,0,0,0,0,0,1,0,0,0
2,ISIC_0015864,0,IP_6724798,60.0,3.40,22.575830,17.128170,37.970460,33.485410,44.174920,...,0,0,0,1,0,0,0,0,1,0
3,ISIC_0015902,0,IP_4111386,65.0,3.22,14.242329,12.164757,21.448144,21.121356,25.746200,...,0,0,0,0,1,1,0,0,0,0
4,ISIC_0024200,0,IP_8313778,55.0,2.73,24.725520,20.057470,26.464900,25.710460,36.217980,...,0,0,0,0,1,1,0,0,0,0


In [14]:
final_df_train.describe()

,target,age_approx,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,...,tbp_lv_location_simple_Left Leg,tbp_lv_location_simple_Right Arm,tbp_lv_location_simple_Right Leg,tbp_lv_location_simple_Torso Back,tbp_lv_location_simple_Torso Front,anatom_site_general_anterior torso,anatom_site_general_head/neck,anatom_site_general_lower extremity,anatom_site_general_posterior torso,anatom_site_general_upper extremity
count,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,...,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000
mean,0.000980,58.026849,3.930827,19.974007,14.919247,28.281706,26.913015,34.786341,30.921279,54.653689,...,0.132337,0.085297,0.124553,0.303950,0.218846,0.218846,0.030035,0.256890,0.303950,0.175927
std,0.031288,13.549664,1.743068,3.999489,3.529384,5.278676,4.482994,5.708469,4.829345,5.520849,...,0.338857,0.279323,0.330211,0.459962,0.413464,0.413464,0.170685,0.436919,0.459962,0.380758
min,0.000000,5.000000,1.000000,-2.487115,-9.080269,-0.730989,9.237066,3.054228,11.846520,-1.574164,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,50.000000,2.840000,17.330821,12.469740,24.704372,23.848125,31.003148,27.658285,51.566273,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,60.000000,3.370000,19.801910,14.713930,28.171570,26.701704,34.822580,30.804893,55.035632,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,70.000000,4.380000,22.304628,17.137175,31.637429,29.679913,38.430298,33.963868,58.298184,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
max,1.000000,85.000000,28.400000,48.189610,37.021680,54.306900,48.372700,58.765170,54.305290,105.875784,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:

gc.collect()

0